Ce fichier Jupyter représente un axe d'amélioration du projet d'Analyse de Données.

Ici nous allons utiliser la méthode d'optimisation de portefeuille de Markovitz (voir source projet principal), afin d'optimiser l'impact de l'elo du stress et de jouer à domicile.

Nous faisons cette étude sur la période 2013_2022 car nous devons vérifier nos pondérations sur les résultats de la saison 2022 (ce qui n'aurait pas été possible sur la saison 2023 car nous n'avons pas les résultats).

Nous avons ajouté toutes les valeurs dont nous allions avoir besoin, l'élo dans un premier temps, l'impact de jouer à domicile et enfin nous avons du ajouter tous les matchs hors championnats joués par les équipes pour calculer le temps entre chaque match, donnant un facteur "stress" qui correspond au stress de l'exercice physique sur les corps.

Enfinn nous avons généré un très grand nombre de "portefeuilles" et avons pris celui avec les meilleurs résultats afin de trouver la bonne pondération

Un facteur d'amélioration serait comme mentionné dans le projet principal de recalculer les paramètres avant chaque match et pas avant chaque saison. Enfin le facteur "stress" que nous avons choisi "exponentiel" qui pourrait être d'une autre forme(voir propositions stress_simple, stress_carré)

Nous vous remercions pour votre attention sur ce projet, si possible nous aimerions avoir un contre rendu de votre part afin de pouvoir améliorer notre précision et si possible à terme utiliser notre système de prédiction.

Cordialement

Robin SIODLAK, Nicolas BIGEARD

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from datetime import datetime

In [11]:
data= pd.read_csv('matchs_2013_2022.csv')
# je veux voir les saisons 2013-2021

data = data[data['season'] != 2022]
print(data)
data1= pd.read_csv('matchs_2013_2022.csv')
# je veux voir la saison 2022

data1 = data1[data1['season'] == 2022]
print(data1)

      Unnamed: 0  game_id  season         round        date  home_club_id  \
0             12  2330874    2013   8. Matchday  2013-09-28           618   
1             13  2330880    2013  10. Matchday  2013-10-19         14171   
2             14  2330916    2013  15. Matchday  2013-11-30          1423   
3             15  2331018    2013  21. Matchday  2014-01-18           273   
4             16  2331034    2013  24. Matchday  2014-02-08          1147   
...          ...      ...     ...           ...         ...           ...   
4073       66083  3199491    2019  14. Matchday  2019-11-24            40   
4074       67000  3394982    2020  36. Matchday  2021-05-09           667   
4075       67467  3047720    2018  20. Matchday  2019-01-13           415   
4076       67720  3199559    2019  21. Matchday  2020-01-25           162   
4077       67721  3394685    2020  13. Matchday  2020-12-06            40   

      away_club_id  home_club_goals  away_club_goals  home_club_position  .

In [12]:

def expected_score(rating1, rating2):
    return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

def update_ratings(rating1, rating2, result, goal_diff, k=20):
    expected1 = expected_score(rating1, rating2)
    diff_ELO = abs(rating1 - rating2)
    multiplier = math.log(abs(goal_diff) + 1) * (2.2 / (diff_ELO * 0.001 + 2.2))
    k_adjusted = k * multiplier
    rating1 += k_adjusted * (result - expected1)
    rating2 += k_adjusted * (1 - result - expected1)
    return rating1, rating2

def calculate_elo(data, years):
    try:
        # Vérifier que 'season' est un entier
        data['season'] = data['season'].astype(int)
        current_year = data['season'].max()
        year_limit = current_year - years

        # Filtrer les données pour les 'years' dernières années
        filtered_data = data[data['season'] > year_limit]
        teams = {team: 1500 for team in pd.concat([filtered_data['home_club_name'], filtered_data['away_club_name']]).unique()}

        # Mise à jour des scores ELO pour chaque match filtré
        for _, row in filtered_data.iterrows():
            home_team, away_team = row['home_club_name'], row['away_club_name']
            result = row['results']
            goal_diff = row['home_club_goals'] - row['away_club_goals']
            home_rating, away_rating = teams[home_team], teams[away_team]
            new_home_rating, new_away_rating = update_ratings(home_rating, away_rating, result, goal_diff)
            teams[home_team], teams[away_team] = new_home_rating, new_away_rating

        # Affichage des scores ELO finaux triés
        sorted_teams = sorted(teams.items(), key=lambda item: item[1], reverse=True)
        for team, rating in sorted_teams:
            print(f"{team}: {rating}")
        return teams
    except Exception as e:
        print(f"An error occurred: {e}")


# Calcul des scores ELO pour les 'y' dernières années
y = 3 
calculate_elo(data, y)


Paris Saint-Germain Football Club: 2577.1840977323304
Lille Olympique Sporting Club Lille Métropole: 1997.6778892363081
Olympique Lyonnais: 1957.69605770213
Association sportive de Monaco Football Club: 1936.5408825111485
Olympique de Marseille: 1871.3494137661628
Stade Rennais Football Club: 1841.6454899490977
Racing Club de Lens: 1665.2210483841766
Olympique Gymnaste Club Nice Côte d'Azur: 1635.1322889417308
Racing Club de Strasbourg Alsace: 1572.3217337186102
Stade de Reims: 1467.8555622653448
Football Club de Nantes: 1460.4073635947439
Montpellier Hérault Sport Club: 1410.0605234833783
ESTAC Troyes: 1384.9463876849393
Amiens SC: 1323.8761930320397
Clermont Foot 63: 1281.7386069300544
Stade brestois 29: 1269.8261018106114
FC Girondins Bordeaux: 1191.6777737136267
Toulouse Football Club: 1152.4624713108387
Football Club de Metz: 1142.6293798178265
Nîmes Olympique: 1140.260811999624
Angers SCO: 1135.0443735299687
Football Club Lorient-Bretagne Sud: 1125.037822128281
AS Saint-Étienne: 

{'Football Club de Metz': 1142.6293798178265,
 'Association sportive de Monaco Football Club': 1936.5408825111485,
 'AS Saint-Étienne': 1028.2815426350771,
 'Montpellier Hérault Sport Club': 1410.0605234833783,
 'FC Girondins Bordeaux': 1191.6777737136267,
 'Football Club de Nantes': 1460.4073635947439,
 "Olympique Gymnaste Club Nice Côte d'Azur": 1635.1322889417308,
 'Angers SCO': 1135.0443735299687,
 'Stade de Reims': 1467.8555622653448,
 'Olympique Lyonnais': 1957.69605770213,
 'Racing Club de Lens': 1665.2210483841766,
 'Lille Olympique Sporting Club Lille Métropole': 1997.6778892363081,
 'ESTAC Troyes': 1384.9463876849393,
 'Stade Rennais Football Club': 1841.6454899490977,
 'Olympique de Marseille': 1871.3494137661628,
 'Paris Saint-Germain Football Club': 2577.1840977323304,
 'Nîmes Olympique': 1140.260811999624,
 'Dijon FCO': 990.6065003267564,
 'Stade brestois 29': 1269.8261018106114,
 'Racing Club de Strasbourg Alsace': 1572.3217337186102,
 'Toulouse Football Club': 1152.4624

In [13]:
# Calculer le nombre de matchs à domicile et à l'extérieur
home_matches = data.groupby('home_club_name').size()
away_matches = data.groupby('away_club_name').size()

# Calculer le nombre de victoires à domicile et à l'extérieur
home_wins = data[data['results'] == 1].groupby('home_club_name').size()
away_wins = data[data['results'] == -1].groupby('away_club_name').size()

# Calculer les pourcentages de victoires à domicile et à l'extérieur
home_win_percentage = (home_wins / home_matches) * 100
away_win_percentage = (away_wins / away_matches) * 100
impact_home_percentage = home_win_percentage - away_win_percentage

# Afficher les résultats
print("Home Win Percentage:")
print(home_win_percentage)
print("\nAway Win Percentage:")
print(away_win_percentage)
print("\nImpact of Playing at Home:")
print(impact_home_percentage)
home_games = data[data['home_club_name'] == 'NOM_DE_LEQUIPE']


Home Win Percentage:
home_club_name
AC Ajaccio                                       26.315789
AS Nancy-Lorraine                                31.578947
AS Saint-Étienne                                 46.739130
Amiens SC                                        34.615385
Angers SCO                                       35.658915
Association sportive de Monaco Football Club     59.036145
Clermont Foot 63                                 21.052632
Dijon FCO                                        34.444444
EA Guingamp                                      40.350877
ESTAC Troyes                                     25.000000
FC Girondins Bordeaux                            41.847826
FC Sochaux-Montbéliard                           36.842105
Football Club Lorient-Bretagne Sud               40.601504
Football Club de Metz                            26.605505
Football Club de Nantes                          40.361446
GFC Ajaccio                                      26.315789
Lille Olympique Spor

In [18]:
def extract_match_dates(df):
    # Préparer un dictionnaire pour collecter les dates de match pour chaque club
    match_dates = {}
    
    # Traiter chaque match dans le DataFrame
    for index, row in df.iterrows():
        home_club = row['home_club_name']
        away_club = row['away_club_name']
        match_date = row['date'] 
        
        # Ajouter la date au club à domicile
        if home_club in match_dates:
            match_dates[home_club].add(match_date)
        else:
            match_dates[home_club] = {match_date}
        
        # Ajouter la date au club à l'extérieur
        if away_club in match_dates:
            match_dates[away_club].add(match_date)
        else:
            match_dates[away_club] = {match_date}
    
    # Convertir les ensembles de dates en listes triées
    for club in match_dates:
        match_dates[club] = sorted(match_dates[club])
    
    return match_dates

# Utiliser la fonction pour obtenir les dates de match pour la saison 2022
match_dates_2022 = extract_match_dates(data1)
print(match_dates_2022)


{'Montpellier Hérault Sport Club': ['2022-08-07', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 'AJ Auxerre': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-14', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-17', '2023-02-26', '2023-03-03', '2023-03-11', '2023-03-19', '2023-04-01', '2023-04-0

In [15]:
# Define the initial teams dictionary with ELO scores
teams = {'Football Club de Metz': 1142.6293798178265,
 'Association sportive de Monaco Football Club': 1936.5408825111485,
 'AS Saint-Étienne': 1028.2815426350771,
 'Montpellier Hérault Sport Club': 1410.0605234833783,
 'FC Girondins Bordeaux': 1191.6777737136267,
 'Football Club de Nantes': 1460.4073635947439,
 "Olympique Gymnaste Club Nice Côte d'Azur": 1635.1322889417308,
 'Angers SCO': 1135.0443735299687,
 'Stade de Reims': 1467.8555622653448,
 'Olympique Lyonnais': 1957.69605770213,
 'Racing Club de Lens': 1665.2210483841766,
 'Lille Olympique Sporting Club Lille Métropole': 1997.6778892363081,
 'ESTAC Troyes': 1384.9463876849393,
 'Stade Rennais Football Club': 1841.6454899490977,
 'Olympique de Marseille': 1871.3494137661628,
 'Paris Saint-Germain Football Club': 2577.1840977323304,
 'Nîmes Olympique': 1140.260811999624,
 'Dijon FCO': 990.6065003267564,
 'Stade brestois 29': 1269.8261018106114,
 'Racing Club de Strasbourg Alsace': 1572.3217337186102,
 'Toulouse Football Club': 1152.4624713108387,
 'Football Club Lorient-Bretagne Sud': 1125.037822128281,
 'Clermont Foot 63': 1281.7386069300544,
 'Amiens SC': 1323.8761930320397}

# Define the modifier values for each club
modificateur = {
    'AC Ajaccio': 18.421053,
    'AS Nancy-Lorraine': 15.789474,
    'AS Saint-Étienne': 14.847239,
    'Amiens SC': 21.153846,
    'Angers SCO': 10.462064,
    'Association sportive de Monaco Football Club': 11.445783,
    'Clermont Foot 63': -5.263158,
    'Dijon FCO': 23.333333,
    'EA Guingamp': 21.052632,
    'ESTAC Troyes': 11.842105,
    'FC Girondins Bordeaux': 16.579009,
    'FC Sochaux-Montbéliard': 21.052632,
    'Football Club Lorient-Bretagne Sud': 21.804511,
    'Football Club de Metz': 7.339450,
    'Football Club de Nantes': 13.855422,
    'GFC Ajaccio': 10.526316,
    'Lille Olympique Sporting Club Lille Métropole': 14.112903,
    'Montpellier Hérault Sport Club': 19.459459,
    'Nîmes Olympique': 9.615385,
    "Olympique Gymnaste Club Nice Côte d'Azur": 19.535998,
    'Olympique Lyonnais': 14.371328,
    'Olympique de Marseille': 12.972973,
    'Paris Saint-Germain Football Club': 14.245006,
    'Racing Club de Lens': 5.263158,
    'Racing Club de Strasbourg Alsace': 12.659176,
    'SC Bastia': 28.421053,
    'SM Caen': 13.684211,
    'Stade Rennais Football Club': 12.972973,
    'Stade brestois 29': 16.901408,
    'Stade de Reims': 11.564626,
    'Thonon Évian Grand Genève FC': 14.035088,
    'Toulouse Football Club': 13.957793,
    'Valenciennes FC': 18.421053
}


# Update the teams dictionary to include both ELO and modifier values
for club in teams:
    if club in modificateur:
        teams[club] = {'elo': teams[club], 'modificateur': modificateur[club]}

# Print the updated teams dictionary
for club, details in teams.items():
    print(club, ' : ', details)
    


Football Club de Metz  :  {'elo': 1142.6293798178265, 'modificateur': 7.33945}
Association sportive de Monaco Football Club  :  {'elo': 1936.5408825111485, 'modificateur': 11.445783}
AS Saint-Étienne  :  {'elo': 1028.2815426350771, 'modificateur': 14.847239}
Montpellier Hérault Sport Club  :  {'elo': 1410.0605234833783, 'modificateur': 19.459459}
FC Girondins Bordeaux  :  {'elo': 1191.6777737136267, 'modificateur': 16.579009}
Football Club de Nantes  :  {'elo': 1460.4073635947439, 'modificateur': 13.855422}
Olympique Gymnaste Club Nice Côte d'Azur  :  {'elo': 1635.1322889417308, 'modificateur': 19.535998}
Angers SCO  :  {'elo': 1135.0443735299687, 'modificateur': 10.462064}
Stade de Reims  :  {'elo': 1467.8555622653448, 'modificateur': 11.564626}
Olympique Lyonnais  :  {'elo': 1957.69605770213, 'modificateur': 14.371328}
Racing Club de Lens  :  {'elo': 1665.2210483841766, 'modificateur': 5.263158}
Lille Olympique Sporting Club Lille Métropole  :  {'elo': 1997.6778892363081, 'modificate

In [23]:
match_dates = {
 'Montpellier Hérault Sport Club': ['2023-01-06', '2022-08-07', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'],
 'AJ Auxerre': ["2023-01-08", "2023-01-22", "2023-02-08", '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-14', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-17', '2023-02-26', '2023-03-03', '2023-03-11', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-16', '2023-04-22', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'],
 'Football Club de Nantes': ["2023-01-07", "2023-01-22", "2023-02-08", "2023-04-05", "2023-04-29", '2022-08-07', '2022-08-12', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-04', '2023-03-12', '2023-03-17', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-05-03', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
 'Clermont Foot 63': ["2023-01-07", '2022-08-06', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-12', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'Lille Olympique Sporting Club Lille Métropole': ["2023-01-08", "2023-01-22", "2023-02-08", '2022-08-07', '2022-08-12', '2022-08-21', '2022-08-26', '2022-08-31', '2022-09-04', '2022-09-10', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-14', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-19', '2023-02-24', '2023-03-04', '2023-03-10', '2023-03-18', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-22', '2023-04-29', '2023-05-06', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
 'Angers SCO': ["2023-01-06", "2023-01-21", "2023-02-08", '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-18', '2022-09-30', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-18', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-18', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-21', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'Paris Saint-Germain Football Club': ["2023-01-06", "2023-01-23", "2023-02-08", "2022-09-06", "2022-09-14", "2022-10-05", "2022-10-11", "2022-10-25", "2022-11-02", "2023-02-14", "2023-03-08", "2022-07-31", '2022-08-06', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-10', '2022-09-18', '2022-10-01', '2022-10-08', '2022-10-16', '2022-10-21', '2022-10-29', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-04', '2023-03-11', '2023-03-19', '2023-04-02', '2023-04-08', '2023-04-15', '2023-04-21', '2023-04-30', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'Racing Club de Lens': ["2023-01-07", "2023-01-22", "2023-02-09", "2023-03-01", '2022-08-07', '2022-08-14', '2022-08-20', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-09', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-28', '2022-11-05', '2022-11-12', '2022-12-29', '2023-01-01', '2023-01-11', '2023-01-14', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-04', '2023-03-12', '2023-03-18', '2023-04-01', '2023-04-07', '2023-04-15', '2023-04-22', '2023-05-02', '2023-05-06', '2023-05-12', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'Olympique de Marseille': ["2023-01-07", "2023-01-20", "2023-02-08", "2023-03-01", "2022-09-07", "2022-09-13", "2022-10-04", "2022-10-12", "2022-10-26", "2022-11-01", '2022-08-07', '2022-08-14', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-10', '2022-09-18', '2022-09-30', '2022-10-08', '2022-10-16', '2022-10-22', '2022-10-29', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-11', '2023-01-14', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
 'Stade de Reims': ["2023-01-08", "2023-01-21", "2022-02-08", '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-08', '2022-10-15', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-18', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-15', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-12', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'AC Ajaccio': ["2023-01-08", "2023-01-21", '2022-08-05', '2022-08-14', '2022-08-21', '2022-08-26', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-08', '2022-10-16', '2022-10-21', '2022-10-30', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-10', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-29', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'Association sportive de Monaco Football Club': ["2023-01-07", "2022-08-02", "2022-08-09", '2022-08-06', '2022-08-13', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-15', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-22', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-19', '2023-05-27', '2023-06-03'], 
 'Football Club Lorient-Bretagne Sud': ["2023-01-08", "2023-01-21", "2023-02-09", '2022-08-07', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-07', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-01', '2023-01-11', '2023-01-14', '2023-01-27', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
 "Olympique Gymnaste Club Nice Côte d'Azur": ["2023-01-07", '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-01', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-11', '2022-12-29', '2023-01-02', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-10', '2023-02-18', '2023-02-26', '2023-03-03', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'Racing Club de Strasbourg Alsace': ["2023-01-06", '2022-08-06', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-01', '2022-10-09', '2022-10-14', '2022-10-23', '2022-10-29', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-11', '2023-01-14', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-18', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-07', '2023-04-16', '2023-04-23', '2023-04-28', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'Olympique Lyonnais': ["2023-01-07", "2023-01-21", "2023-02-08", "2023-02-28", "2023-04-05", '2022-08-05', '2022-08-19', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-07', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-07', '2022-10-16', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-11', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-14', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-17', '2023-02-25', '2023-03-05', '2023-03-10', '2023-03-17', '2023-04-02', '2023-04-09', '2023-04-14', '2023-04-23', '2023-04-28', '2023-05-07', '2023-05-14', '2023-05-19', '2023-05-27', '2023-06-03'], 
 'Stade brestois 29': ["2023-01-07", "2023-01-22", '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-10', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-24', '2023-03-05', '2023-03-11', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-05-03', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'Toulouse Football Club': ["2023-01-08", "2023-01-21", "2023-02-08", "2023-03-01", "2023-04-05", "2023-04-05", '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-07', '2022-10-16', '2022-10-23', '2022-10-28', '2022-11-06', '2022-11-12', '2022-12-29', '2023-01-01', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-18', '2023-04-02', '2023-04-09', '2023-04-14', '2023-04-23', '2023-05-02', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'ESTAC Troyes': ["2023-01-08", '2022-08-07', '2022-08-14', '2022-08-19', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-09', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-29', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-11', '2023-01-15', '2023-01-28', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
 'Stade Rennais Football Club': ["2023-01-07", "2023-01-20", "2022-09-08", "2022-09-15", "2022-10-06", "2022-10-13", "2022-10-27", "2022-11-03", "2023-02-16", "2023-02-23", '2022-08-07', '2022-08-13', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-01', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-12', '2022-12-29', '2023-01-02', '2023-01-11', '2023-01-15', '2023-01-27', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-11', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-15', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03']}

for club, dates in match_dates.items():
    match_dates[club] = sorted(dates, key=lambda date: datetime.strptime(date, "%Y-%m-%d"))

print(match_dates)

{'Montpellier Hérault Sport Club': ['2022-08-07', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 'AJ Auxerre': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-14', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-17', '2023-02-26', '2023-03-0

In [24]:
#match pour différents clubs
match_dates = {'Montpellier Hérault Sport Club': ['2022-08-07', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'AJ Auxerre': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-14', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-17', '2023-02-26', '2023-03-03', '2023-03-11', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-16', '2023-04-22', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Football Club de Nantes': ['2022-08-07', '2022-08-12', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-04', '2023-03-12', '2023-03-17', '2023-04-02', '2023-04-05', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-29', '2023-05-03', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'Clermont Foot 63': ['2022-08-06', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-12', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Lille Olympique Sporting Club Lille Métropole': ['2022-08-07', '2022-08-12', '2022-08-21', '2022-08-26', '2022-08-31', '2022-09-04', '2022-09-10', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-14', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-12', '2023-02-19', '2023-02-24', '2023-03-04', '2023-03-10', '2023-03-18', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-22', '2023-04-29', '2023-05-06', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'Angers SCO': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-18', '2022-09-30', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-18', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-18', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-21', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Paris Saint-Germain Football Club': ['2022-07-31', '2022-08-06', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-06', '2022-09-10', '2022-09-14', '2022-09-18', '2022-10-01', '2022-10-05', '2022-10-08', '2022-10-11', '2022-10-16', '2022-10-21', '2022-10-25', '2022-10-29', '2022-11-02', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-23', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-11', '2023-02-14', '2023-02-19', '2023-02-26', '2023-03-04', '2023-03-08', '2023-03-11', '2023-03-19', '2023-04-02', '2023-04-08', '2023-04-15', '2023-04-21', '2023-04-30', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Racing Club de Lens': ['2022-08-07', '2022-08-14', '2022-08-20', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-09', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-28', '2022-11-05', '2022-11-12', '2022-12-29', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-14', '2023-01-22', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-09', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-01', '2023-03-04', '2023-03-12', '2023-03-18', '2023-04-01', '2023-04-07', '2023-04-15', '2023-04-22', '2023-05-02', '2023-05-06', '2023-05-12', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Olympique de Marseille': ['2022-08-07', '2022-08-14', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-07', '2022-09-10', '2022-09-13', '2022-09-18', '2022-09-30', '2022-10-04', '2022-10-08', '2022-10-12', '2022-10-16', '2022-10-22', '2022-10-26', '2022-10-29', '2022-11-01', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-07', '2023-01-11', '2023-01-14', '2023-01-20', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-01', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'Stade de Reims': ['2022-02-08', '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-08', '2022-10-15', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-18', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-15', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-12', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'AC Ajaccio': ['2022-08-05', '2022-08-14', '2022-08-21', '2022-08-26', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-08', '2022-10-16', '2022-10-21', '2022-10-30', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-10', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-29', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Association sportive de Monaco Football Club': ['2022-08-02', '2022-08-06', '2022-08-09', '2022-08-13', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-22', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-19', '2023-05-27', '2023-06-03'], 
               'Football Club Lorient-Bretagne Sud': ['2022-08-07', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-07', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-14', '2023-01-21', '2023-01-27', '2023-02-01', '2023-02-05', '2023-02-09', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               "Olympique Gymnaste Club Nice Côte d'Azur": ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-01', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-11', '2022-12-29', '2023-01-02', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-10', '2023-02-18', '2023-02-26', '2023-03-03', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Racing Club de Strasbourg Alsace': ['2022-08-06', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-01', '2022-10-09', '2022-10-14', '2022-10-23', '2022-10-29', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-06', '2023-01-11', '2023-01-14', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-18', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-07', '2023-04-16', '2023-04-23', '2023-04-28', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Olympique Lyonnais': ['2022-08-05', '2022-08-19', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-07', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-07', '2022-10-16', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-11', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-14', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-12', '2023-02-17', '2023-02-25', '2023-02-28', '2023-03-05', '2023-03-10', '2023-03-17', '2023-04-02', '2023-04-05', '2023-04-09', '2023-04-14', '2023-04-23', '2023-04-28', '2023-05-07', '2023-05-14', '2023-05-19', '2023-05-27', '2023-06-03'], 
               'Stade brestois 29': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-10', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-24', '2023-03-05', '2023-03-11', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-05-03', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Toulouse Football Club': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-07', '2022-10-16', '2022-10-23', '2022-10-28', '2022-11-06', '2022-11-12', '2022-12-29', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-01', '2023-03-05', '2023-03-12', '2023-03-18', '2023-04-02', '2023-04-05', '2023-04-05', '2023-04-09', '2023-04-14', '2023-04-23', '2023-05-02', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'ESTAC Troyes': ['2022-08-07', '2022-08-14', '2022-08-19', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-09', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-29', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-28', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Stade Rennais Football Club': ['2022-08-07', '2022-08-13', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-08', '2022-09-11', '2022-09-15', '2022-09-18', '2022-10-01', '2022-10-06', '2022-10-09', '2022-10-13', '2022-10-16', '2022-10-23', '2022-10-27', '2022-10-30', '2022-11-03', '2022-11-06', '2022-11-12', '2022-12-29', '2023-01-02', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-20', '2023-01-27', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-16', '2023-02-19', '2023-02-23', '2023-02-26', '2023-03-05', '2023-03-11', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-15', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03']}


def process_match_dates(match_dates):
    results = {}
    stress_simple = {}
    stress_square_inverse = {}
    stress_exponential_inverse = {}
    for club, dates in match_dates.items():
        # Conversion des dates en datetime et suppression des doublons
        unique_dates = pd.to_datetime(list(set(dates))).sort_values()

        # Stockage des dates triées en format string
        results[club] = unique_dates.strftime('%Y-%m-%d').tolist()
        
        # Calcul des différences en jours entre les dates consécutives
        if len(unique_dates) > 1:
            day_diffs = unique_dates.diff().days[1:]  # Exclure le premier élément NaN
            print(f"Dates de match pour {club}:", results[club])
            print(f'Différences en jours pour {club}: {list(day_diffs.values)}')
            # Calcul de stress
            stress_simple[club] = [1 / day for day in day_diffs]
            stress_square_inverse[club] = [1 / (day ** 2) for day in day_diffs]
            stress_exponential_inverse[club] = [np.exp(-day) for day in day_diffs]
            print(f"Stress Simple pour {club}:", stress_simple[club])
            print(f"Stress Carré Inverse pour {club}:", stress_square_inverse[club])
            print(f"Stress Exponentiel Inverse pour {club}:", stress_exponential_inverse[club])
            print()
        else:
            print(f'Pas de différences à calculer pour {club} (une seule date).')
            stress_simple[club] = []
            stress_square_inverse[club] = []
            stress_exponential_inverse[club] = []
    
    return results, stress_simple, stress_square_inverse, stress_exponential_inverse




## Appel de la fonction et affichage des résultats


final_dates, stress_simple, stress_square_inverse, stress_exponential_inverse = process_match_dates(match_dates)
#print(len(final_dates['Paris Saint-Germain Football Club']))
#print(len(final_dates['Association sportive de Monaco Football Club']))



Dates de match pour Montpellier Hérault Sport Club: ['2022-08-07', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03']
Différences en jours pour Montpellier Hérault Sport Club: [6.0, 8.0, 7.0, 3.0, 4.0, 7.0, 6.0, 15.0, 7.0, 6.0, 7.0, 8.0, 7.0, 7.0, 46.0, 4.0, 4.0, 5.0, 4.0, 14.0, 3.0, 4.0, 7.0, 7.0, 6.0, 8.0, 7.0, 7.0, 12.0, 9.0, 7.0, 7.0, 7.0, 7.0, 7.0, 6.0, 7.0, 7.0]
Stress Simple pour Montpellier Hérault Sport Club: [0.16666666666666666, 0.125, 0.14285714285714285, 0.3333333333333333, 0.25, 0.14285714285714285, 

In [25]:
import pandas as pd
import numpy as np

# Données des dates de match pour différents clubs
match_dates = {'Montpellier Hérault Sport Club': ['2022-08-07', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'AJ Auxerre': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-14', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-17', '2023-02-26', '2023-03-03', '2023-03-11', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-16', '2023-04-22', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Football Club de Nantes': ['2022-08-07', '2022-08-12', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-04', '2023-03-12', '2023-03-17', '2023-04-02', '2023-04-05', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-29', '2023-05-03', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'Clermont Foot 63': ['2022-08-06', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-12', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Lille Olympique Sporting Club Lille Métropole': ['2022-08-07', '2022-08-12', '2022-08-21', '2022-08-26', '2022-08-31', '2022-09-04', '2022-09-10', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-14', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-12', '2023-02-19', '2023-02-24', '2023-03-04', '2023-03-10', '2023-03-18', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-22', '2023-04-29', '2023-05-06', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'Angers SCO': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-18', '2022-09-30', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-18', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-18', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-21', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Paris Saint-Germain Football Club': ['2022-07-31', '2022-08-06', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-06', '2022-09-10', '2022-09-14', '2022-09-18', '2022-10-01', '2022-10-05', '2022-10-08', '2022-10-11', '2022-10-16', '2022-10-21', '2022-10-25', '2022-10-29', '2022-11-02', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-23', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-11', '2023-02-14', '2023-02-19', '2023-02-26', '2023-03-04', '2023-03-08', '2023-03-11', '2023-03-19', '2023-04-02', '2023-04-08', '2023-04-15', '2023-04-21', '2023-04-30', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Racing Club de Lens': ['2022-08-07', '2022-08-14', '2022-08-20', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-09', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-28', '2022-11-05', '2022-11-12', '2022-12-29', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-14', '2023-01-22', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-09', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-01', '2023-03-04', '2023-03-12', '2023-03-18', '2023-04-01', '2023-04-07', '2023-04-15', '2023-04-22', '2023-05-02', '2023-05-06', '2023-05-12', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Olympique de Marseille': ['2022-08-07', '2022-08-14', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-07', '2022-09-10', '2022-09-13', '2022-09-18', '2022-09-30', '2022-10-04', '2022-10-08', '2022-10-12', '2022-10-16', '2022-10-22', '2022-10-26', '2022-10-29', '2022-11-01', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-07', '2023-01-11', '2023-01-14', '2023-01-20', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-01', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'Stade de Reims': ['2022-02-08', '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-08', '2022-10-15', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-18', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-15', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-12', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'AC Ajaccio': ['2022-08-05', '2022-08-14', '2022-08-21', '2022-08-26', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-08', '2022-10-16', '2022-10-21', '2022-10-30', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-10', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-29', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Association sportive de Monaco Football Club': ['2022-08-02', '2022-08-06', '2022-08-09', '2022-08-13', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-22', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-19', '2023-05-27', '2023-06-03'], 
               'Football Club Lorient-Bretagne Sud': ['2022-08-07', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-07', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-14', '2023-01-21', '2023-01-27', '2023-02-01', '2023-02-05', '2023-02-09', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               "Olympique Gymnaste Club Nice Côte d'Azur": ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-01', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-11', '2022-12-29', '2023-01-02', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-10', '2023-02-18', '2023-02-26', '2023-03-03', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Racing Club de Strasbourg Alsace': ['2022-08-06', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-01', '2022-10-09', '2022-10-14', '2022-10-23', '2022-10-29', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-06', '2023-01-11', '2023-01-14', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-18', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-07', '2023-04-16', '2023-04-23', '2023-04-28', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Olympique Lyonnais': ['2022-08-05', '2022-08-19', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-07', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-07', '2022-10-16', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-11', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-14', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-12', '2023-02-17', '2023-02-25', '2023-02-28', '2023-03-05', '2023-03-10', '2023-03-17', '2023-04-02', '2023-04-05', '2023-04-09', '2023-04-14', '2023-04-23', '2023-04-28', '2023-05-07', '2023-05-14', '2023-05-19', '2023-05-27', '2023-06-03'], 
               'Stade brestois 29': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-10', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-24', '2023-03-05', '2023-03-11', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-05-03', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Toulouse Football Club': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-07', '2022-10-16', '2022-10-23', '2022-10-28', '2022-11-06', '2022-11-12', '2022-12-29', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-01', '2023-03-05', '2023-03-12', '2023-03-18', '2023-04-02', '2023-04-05', '2023-04-05', '2023-04-09', '2023-04-14', '2023-04-23', '2023-05-02', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'ESTAC Troyes': ['2022-08-07', '2022-08-14', '2022-08-19', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-09', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-29', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-28', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Stade Rennais Football Club': ['2022-08-07', '2022-08-13', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-08', '2022-09-11', '2022-09-15', '2022-09-18', '2022-10-01', '2022-10-06', '2022-10-09', '2022-10-13', '2022-10-16', '2022-10-23', '2022-10-27', '2022-10-30', '2022-11-03', '2022-11-06', '2022-11-12', '2022-12-29', '2023-01-02', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-20', '2023-01-27', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-16', '2023-02-19', '2023-02-23', '2023-02-26', '2023-03-05', '2023-03-11', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-15', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03']}

def process_exponential_stress(match_dates):
    stress_exponential_inverse = {}
    for club, dates in match_dates.items():
        unique_dates = pd.to_datetime(list(set(dates))).sort_values()
        if len(unique_dates) > 1:
            day_diffs = unique_dates.diff().days[1:]  # Exclude the first NaN element
            stress_exponential_inverse[club] = [np.exp(-day)*100000 for day in day_diffs]
        else:
            stress_exponential_inverse[club] = []
    return stress_exponential_inverse

# Example usage of stress results
stress_results = process_exponential_stress(match_dates)

data = data1

def get_match_details(match_id, stress_results):
    match = data[data['game_id'] == match_id]
    if match.empty:
        return "Le match n'existe pas."

    home_club = match['home_club_name'].iloc[0]
    away_club = match['away_club_name'].iloc[0]

    home_elo = teams.get(home_club, {}).get('elo', 'ELO non disponible')
    away_elo = teams.get(away_club, {}).get('elo', 'ELO non disponible')
    home_modificateur = teams.get(home_club, {}).get('modificateur', 'Modificateur non disponible')
    match_date = pd.to_datetime(match['date'].iloc[0])


    # Get the stress values for the home club
    club_dates = pd.to_datetime(match_dates[home_club]).sort_values()
    match_index = club_dates.get_loc(match_date)
    stress_value_home = stress_results[home_club][match_index] if match_index > 0 else 0
    stress_value_away = stress_results[away_club][match_index] if match_index > 0 else 0


    return {
    'home_club': home_club,
    'away_club': away_club,
    'match_date': match_date.strftime('%Y-%m-%d'),
    'stress_value_home': stress_value_home,
    'stress_value_away': stress_value_away,
    'home_elo': home_elo,
    'away_elo': away_elo,
    'home_modificateur': home_modificateur
}


# Example usage
match_id = int(input("Entrez l'identifiant du match: "))
details = get_match_details(match_id, stress_results)

if isinstance(details, str):
    print(details)
else:
    print(f"Le match {details['home_club']} contre {details['away_club']} le {details['match_date']} a une valeur de stress pour {details['home_club']} de {details['stress_value_home']:.3f}, et pour {details['away_club']} de {details['stress_value_away']:.3f}.")
    print(f"Le match {details['home_club']} : {details['home_elo']} vs {details['away_club']} : {details['away_elo']} a eu lieu le {details['match_date']} à domicile pour {details['home_club']}.")
    print(f"Le modificateur pour {details['home_club']} est de {details['home_modificateur']}.")


Le match Football Club Lorient-Bretagne Sud contre Association sportive de Monaco Football Club le 2023-01-11 a une valeur de stress pour Football Club Lorient-Bretagne Sud de 4978.707, et pour Association sportive de Monaco Football Club de 247.875.
Le match Football Club Lorient-Bretagne Sud : 1125.037822128281 vs Association sportive de Monaco Football Club : 1936.5408825111485 a eu lieu le 2023-01-11 à domicile pour Football Club Lorient-Bretagne Sud.
Le modificateur pour Football Club Lorient-Bretagne Sud est de 21.804511.


In [43]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Préparation des données de match et de stress
data = data1

# Dictionnaire pour les données de match et de stress
teams = {
'Football Club de Metz'  :  {'elo': 1142.6293798178265, 'modificateur': 7.33945},
'Association sportive de Monaco Football Club'  :  {'elo': 1936.5408825111485, 'modificateur': 11.445783},
'AS Saint-Étienne'  :  {'elo': 1028.2815426350771, 'modificateur': 14.847239},
'Montpellier Hérault Sport Club'  :  {'elo': 1410.0605234833783, 'modificateur': 19.459459},
'FC Girondins Bordeaux' :  {'elo': 1191.6777737136267, 'modificateur': 16.579009},
'Football Club de Nantes' :  {'elo': 1460.4073635947439, 'modificateur': 13.855422},
"Olympique Gymnaste Club Nice Côte d'Azur" :  {'elo': 1635.1322889417308, 'modificateur': 19.535998},
'Angers SCO' :  {'elo': 1135.0443735299687, 'modificateur': 10.462064},
'Stade de Reims' :  {'elo': 1467.8555622653448, 'modificateur': 11.564626},
'Olympique Lyonnais' :  {'elo': 1957.69605770213, 'modificateur': 14.371328},
'Racing Club de Lens' :  {'elo': 1665.2210483841766, 'modificateur': 5.263158},
'Lille Olympique Sporting Club Lille Métropole' :  {'elo': 1997.6778892363081, 'modificateur': 14.112903},
'ESTAC Troyes' :  {'elo': 1384.9463876849393, 'modificateur': 11.842105},
'Stade Rennais Football Club' :  {'elo': 1841.6454899490977, 'modificateur': 12.972973},
'Olympique de Marseille' :  {'elo': 1871.3494137661628, 'modificateur': 12.972973},
'Paris Saint-Germain Football Club' :  {'elo': 2577.1840977323304, 'modificateur': 14.245006},
'Nîmes Olympique' :  {'elo': 1140.260811999624, 'modificateur': 9.615385},
'Dijon FCO' :  {'elo': 990.6065003267564, 'modificateur': 23.333333},
'Stade brestois 29'  :  {'elo': 1269.8261018106114, 'modificateur': 16.901408},
'Racing Club de Strasbourg Alsace' :  {'elo': 1572.3217337186102, 'modificateur': 12.659176},
'Toulouse Football Club' :  {'elo': 1152.4624713108387, 'modificateur': 13.957793},
'Football Club Lorient-Bretagne Sud' :  {'elo': 1125.037822128281, 'modificateur': 21.804511},
'Clermont Foot 63' :  {'elo': 1281.7386069300544, 'modificateur': -5.263158},
'Amiens SC' :  {'elo': 1323.8761930320397, 'modificateur': 21.153846},
'AC Ajaccio' : {'elo': 1000.0, 'modificateur': 10.0} ,
'AJ Auxerre' : {'elo': 1000.0, 'modificateur': 10.0} ,
}

# Générer les valeurs de stress
def process_exponential_stress(match_dates):
    stress_exponential_inverse = {}
    for club, dates in match_dates.items():
        unique_dates = pd.to_datetime(list(set(dates))).sort_values()
        if len(unique_dates) > 1:
            day_diffs = unique_dates.diff().days[1:]
            stress_exponential_inverse[club] = np.exp(-0.05 * day_diffs) * 100
        else:
            stress_exponential_inverse[club] = np.array([0])
    return stress_exponential_inverse

match_dates = {'Montpellier Hérault Sport Club': ['2022-08-07', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'AJ Auxerre': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-14', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-17', '2023-02-26', '2023-03-03', '2023-03-11', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-16', '2023-04-22', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Football Club de Nantes': ['2022-08-07', '2022-08-12', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-04', '2023-03-12', '2023-03-17', '2023-04-02', '2023-04-05', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-29', '2023-05-03', '2023-05-07', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'Clermont Foot 63': ['2022-08-06', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-12', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Lille Olympique Sporting Club Lille Métropole': ['2022-08-07', '2022-08-12', '2022-08-21', '2022-08-26', '2022-08-31', '2022-09-04', '2022-09-10', '2022-09-17', '2022-10-02', '2022-10-09', '2022-10-14', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-12', '2023-02-19', '2023-02-24', '2023-03-04', '2023-03-10', '2023-03-18', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-22', '2023-04-29', '2023-05-06', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'Angers SCO': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-11', '2022-09-18', '2022-09-30', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-12', '2023-02-18', '2023-02-25', '2023-03-05', '2023-03-12', '2023-03-18', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-21', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Paris Saint-Germain Football Club': ['2022-07-31', '2022-08-06', '2022-08-13', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-06', '2022-09-10', '2022-09-14', '2022-09-18', '2022-10-01', '2022-10-05', '2022-10-08', '2022-10-11', '2022-10-16', '2022-10-21', '2022-10-25', '2022-10-29', '2022-11-02', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-06', '2023-01-11', '2023-01-15', '2023-01-23', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-11', '2023-02-14', '2023-02-19', '2023-02-26', '2023-03-04', '2023-03-08', '2023-03-11', '2023-03-19', '2023-04-02', '2023-04-08', '2023-04-15', '2023-04-21', '2023-04-30', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Racing Club de Lens': ['2022-08-07', '2022-08-14', '2022-08-20', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-09', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-22', '2022-10-28', '2022-11-05', '2022-11-12', '2022-12-29', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-14', '2023-01-22', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-09', '2023-02-12', '2023-02-19', '2023-02-25', '2023-03-01', '2023-03-04', '2023-03-12', '2023-03-18', '2023-04-01', '2023-04-07', '2023-04-15', '2023-04-22', '2023-05-02', '2023-05-06', '2023-05-12', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Olympique de Marseille': ['2022-08-07', '2022-08-14', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-07', '2022-09-10', '2022-09-13', '2022-09-18', '2022-09-30', '2022-10-04', '2022-10-08', '2022-10-12', '2022-10-16', '2022-10-22', '2022-10-26', '2022-10-29', '2022-11-01', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-07', '2023-01-11', '2023-01-14', '2023-01-20', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-08', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-01', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-31', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-14', '2023-05-20', '2023-05-27', '2023-06-03'], 
               'Stade de Reims': ['2022-02-08', '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-08', '2022-10-15', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-02', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-18', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-15', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-12', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'AC Ajaccio': ['2022-08-05', '2022-08-14', '2022-08-21', '2022-08-26', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-08', '2022-10-16', '2022-10-21', '2022-10-30', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-10', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-29', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Association sportive de Monaco Football Club': ['2022-08-02', '2022-08-06', '2022-08-09', '2022-08-13', '2022-08-20', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-28', '2023-02-01', '2023-02-05', '2023-02-11', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-22', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-19', '2023-05-27', '2023-06-03'], 
               'Football Club Lorient-Bretagne Sud': ['2022-08-07', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-07', '2022-09-11', '2022-09-16', '2022-10-02', '2022-10-09', '2022-10-15', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-29', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-14', '2023-01-21', '2023-01-27', '2023-02-01', '2023-02-05', '2023-02-09', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               "Olympique Gymnaste Club Nice Côte d'Azur": ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-18', '2022-10-01', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-11', '2022-12-29', '2023-01-02', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-10', '2023-02-18', '2023-02-26', '2023-03-03', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-08', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Racing Club de Strasbourg Alsace': ['2022-08-06', '2022-08-14', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-01', '2022-10-09', '2022-10-14', '2022-10-23', '2022-10-29', '2022-11-05', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-06', '2023-01-11', '2023-01-14', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-18', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-02', '2023-04-07', '2023-04-16', '2023-04-23', '2023-04-28', '2023-05-07', '2023-05-13', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Olympique Lyonnais': ['2022-08-05', '2022-08-19', '2022-08-28', '2022-08-31', '2022-09-03', '2022-09-07', '2022-09-11', '2022-09-18', '2022-10-02', '2022-10-07', '2022-10-16', '2022-10-22', '2022-10-30', '2022-11-06', '2022-11-11', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-14', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-12', '2023-02-17', '2023-02-25', '2023-02-28', '2023-03-05', '2023-03-10', '2023-03-17', '2023-04-02', '2023-04-05', '2023-04-09', '2023-04-14', '2023-04-23', '2023-04-28', '2023-05-07', '2023-05-14', '2023-05-19', '2023-05-27', '2023-06-03'], 
               'Stade brestois 29': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-10', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-12-28', '2023-01-01', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-01', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-24', '2023-03-05', '2023-03-11', '2023-03-19', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-05-03', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Toulouse Football Club': ['2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-11', '2022-09-17', '2022-10-02', '2022-10-07', '2022-10-16', '2022-10-23', '2022-10-28', '2022-11-06', '2022-11-12', '2022-12-29', '2023-01-01', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-21', '2023-01-29', '2023-02-01', '2023-02-04', '2023-02-08', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-01', '2023-03-05', '2023-03-12', '2023-03-18', '2023-04-02', '2023-04-05', '2023-04-05', '2023-04-09', '2023-04-14', '2023-04-23', '2023-05-02', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'ESTAC Troyes': ['2022-08-07', '2022-08-14', '2022-08-19', '2022-08-28', '2022-08-31', '2022-09-04', '2022-09-09', '2022-09-18', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-29', '2022-11-04', '2022-11-13', '2022-12-28', '2023-01-02', '2023-01-08', '2023-01-11', '2023-01-15', '2023-01-28', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-07', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03'], 
               'Stade Rennais Football Club': ['2022-08-07', '2022-08-13', '2022-08-21', '2022-08-27', '2022-08-31', '2022-09-04', '2022-09-08', '2022-09-11', '2022-09-15', '2022-09-18', '2022-10-01', '2022-10-06', '2022-10-09', '2022-10-13', '2022-10-16', '2022-10-23', '2022-10-27', '2022-10-30', '2022-11-03', '2022-11-06', '2022-11-12', '2022-12-29', '2023-01-02', '2023-01-07', '2023-01-11', '2023-01-15', '2023-01-20', '2023-01-27', '2023-02-01', '2023-02-04', '2023-02-12', '2023-02-16', '2023-02-19', '2023-02-23', '2023-02-26', '2023-03-05', '2023-03-11', '2023-03-19', '2023-04-01', '2023-04-09', '2023-04-15', '2023-04-23', '2023-04-30', '2023-05-06', '2023-05-14', '2023-05-21', '2023-05-27', '2023-06-03']}

stress_results = process_exponential_stress(match_dates)
def get_stress(club, match_date, stress_data):
    if club in match_dates:
        dates = pd.to_datetime(match_dates[club]).sort_values()
        if match_date in dates.values:
            idx = np.where(dates == match_date)[0][0]
            return stress_data[club][idx] if idx < len(stress_data[club]) else 0
    return 0


data['stress_home'] = data.apply(lambda row: get_stress(row['home_club_name'], pd.to_datetime(row['date']), stress_results), axis=1)
data['stress_away'] = data.apply(lambda row: get_stress(row['away_club_name'], pd.to_datetime(row['date']), stress_results), axis=1)

# Ajouter les données ELO, modificateur, et stress au DataFrame principal
def add_match_details(data):
    data['home_elo'] = data['home_club_name'].map(lambda x: teams[x]['elo'])
    data['away_elo'] = data['away_club_name'].map(lambda x: teams[x]['elo'])
    data['modification1_home'] = data['home_club_name'].map(lambda x: teams[x]['modificateur'])
    data['modification1_away'] = data['away_club_name'].map(lambda x: teams[x]['modificateur'])
    data['stress_home'] = data.apply(lambda row: stress_results[row['home_club_name']][0], axis=1)  # Incorrect fix index
    data['stress_away'] = data.apply(lambda row: stress_results[row['away_club_name']][0], axis=1)

    return data

data = add_match_details(data)

# Fonction pour ajuster l'ELO en fonction des poids
def adjust_elo(data, weight_1, weight_2):
    data['adjusted_elo_home'] = data['home_elo'] + weight_1 * data['modification1_home'] + weight_2 * data['stress_home']
    data['adjusted_elo_away'] = data['away_elo'] + weight_1 * data['modification1_away'] + weight_2 * data['stress_away']
    return data

# Fonction pour simuler le modèle et calculer l'exactitude
def simulate_performance(data, weight_1, weight_2):
    data = adjust_elo(data, weight_1, weight_2)
    X = data[['adjusted_elo_home', 'adjusted_elo_away']]
    y = data['results']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    return accuracy_score(y_test, predictions)

# Fonction pour trouver les meilleurs poids
def optimize_weights(data):
    best_score = 0
    best_weights = (0, 0)
    for _ in range(10000):
        weight_1 = np.random.rand()
        weight_2 = np.random.rand()
        score = simulate_performance(data, weight_1, weight_2)
        if score > best_score:
            best_score = score
            best_weights = (weight_1, weight_2)
    return best_weights, best_score

# Optimiser les poids
best_weights, best_score = optimize_weights(data)
print("Meilleurs poids:", best_weights)
print("Meilleur score:", best_score)

C:\Users\nicob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\nicob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stab

Meilleurs poids: (0.8361653598982715, 0.14441098438741462)
Meilleur score: 0.4649122807017544
